<a href="https://colab.research.google.com/github/gulliyevn/DeepLearningModels/blob/main/TraderPlusOnline3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BTC LSTM Trading Model v3.0 - Multi-Timeframe Ensemble
# Enhanced version with fine-tuning, SMC patterns, and memory efficiency

import numpy as np
import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')

# Try to install yfinance if not available
try:
    import yfinance as yf
except ImportError:
    print("📦 Installing yfinance...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance"])
    import yfinance as yf

# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import gc
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
SAVE_PATH = '/content/drive/MyDrive/KERAS'
API_KEY = '2f88eb1e7f9b49ef884557f27c95bd37'
BASE_URL = 'https://api.twelvedata.com'

# Create directories
os.makedirs(SAVE_PATH, exist_ok=True)
print(f"✅ Save path ready: {SAVE_PATH}")

class BTCDataLoader:
    """Enhanced data loader for multiple timeframes with memory efficiency"""

    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://api.twelvedata.com'

    def test_api_connection(self):
        """Test API connection and limits"""
        url = f"{self.base_url}/api_usage"
        params = {'apikey': self.api_key}

        try:
            response = requests.get(url, params=params, timeout=10)
            usage_data = response.json()
            print(f"📡 API Status: {usage_data}")
            return True
        except Exception as e:
            print(f"⚠️ API test failed: {e}")
            return False

    def fetch_data(self, symbol='BTCUSD', interval='15min', outputsize=5000):
        """Fetch data with enhanced error handling and multiple symbol formats"""

        # Try different symbol formats
        symbols_to_try = [symbol, 'BTC/USD', 'BTCUSD', 'BTC-USD']

        for sym in symbols_to_try:
            print(f"🔄 Trying symbol: {sym}")

            url = f"{self.base_url}/time_series"
            params = {
                'symbol': sym,
                'interval': interval,
                'apikey': self.api_key,
                'outputsize': min(outputsize, 1000),  # Reduce initial request
                'format': 'JSON'
            }

            try:
                response = requests.get(url, params=params, timeout=30)
                print(f"📡 Response status: {response.status_code}")

                data = response.json()
                print(f"📄 Response keys: {list(data.keys())}")

                # Handle different response formats
                if 'values' in data and data['values']:
                    df = pd.DataFrame(data['values'])
                    print(f"✅ Raw data shape: {df.shape}")
                    print(f"📋 Columns: {df.columns.tolist()}")

                    # Check for required columns
                    required_cols = ['datetime', 'open', 'high', 'low', 'close']
                    missing_cols = [col for col in required_cols if col not in df.columns]

                    if missing_cols:
                        print(f"⚠️ Missing columns: {missing_cols}")
                        continue

                    df['datetime'] = pd.to_datetime(df['datetime'])
                    df = df.sort_values('datetime').reset_index(drop=True)

                    # Convert to numeric (handle volume separately)
                    price_cols = ['open', 'high', 'low', 'close']
                    for col in price_cols:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                    # Handle volume (may not exist for crypto)
                    if 'volume' in df.columns:
                        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
                    else:
                        # Create synthetic volume based on price movement
                        df['volume'] = abs(df['close'].pct_change()) * 1000000
                        print("📊 Created synthetic volume column")

                    # Remove rows with NaN prices
                    initial_len = len(df)
                    df = df.dropna(subset=price_cols)
                    if len(df) < initial_len:
                        print(f"🧹 Removed {initial_len - len(df)} rows with NaN prices")

                    if len(df) > 100:  # Minimum viable dataset
                        print(f"✅ {interval} data loaded: {len(df)} records with symbol {sym}")
                        return df
                    else:
                        print(f"⚠️ Too few records: {len(df)}")
                        continue

                elif 'message' in data:
                    print(f"📝 API Message: {data['message']}")
                    if 'limit' in data['message'].lower():
                        print("⏳ API limit reached, waiting...")
                        time.sleep(10)
                        continue
                elif 'status' in data and data['status'] == 'error':
                    print(f"❌ API Error: {data}")
                    continue
                else:
                    print(f"❓ Unexpected response format: {data}")
                    continue

            except requests.exceptions.Timeout:
                print(f"⏰ Timeout for symbol {sym}")
                continue
            except Exception as e:
                print(f"❌ Error with symbol {sym}: {e}")
                continue

        print(f"❌ All symbol formats failed for {interval}")
        return None

    def fetch_fallback_data(self, interval='15min'):
        """Fallback to yfinance if available"""
        try:
            import yfinance as yf
            print(f"🔄 Trying yfinance fallback for {interval}...")

            # Convert interval format
            yf_interval_map = {
                '5min': '5m',
                '15min': '15m',
                '1h': '1h'
            }

            if interval not in yf_interval_map:
                return None

            yf_interval = yf_interval_map[interval]

            # Download BTC data
            ticker = yf.Ticker("BTC-USD")
            df = ticker.history(period="1y", interval=yf_interval)

            if df.empty:
                return None

            # Convert to our format
            df = df.reset_index()
            df.columns = [col.lower() for col in df.columns]

            # Rename columns to match our format
            if 'date' in df.columns:
                df = df.rename(columns={'date': 'datetime'})
            elif 'datetime' not in df.columns:
                df['datetime'] = df.index

            # Ensure volume exists
            if 'volume' not in df.columns:
                df['volume'] = abs(df['close'].pct_change()) * 1000000

            print(f"✅ Fallback {interval} data loaded: {len(df)} records")
            return df

        except ImportError:
            print("⚠️ yfinance not available")
            return None
        except Exception as e:
            print(f"❌ Fallback failed: {e}")
            return None

    def generate_synthetic_data(self, interval='15min', periods=2000):
        """Generate synthetic BTC data for testing"""
        print(f"🎲 Generating synthetic {interval} data...")

        # Create date range
        if interval == '5min':
            freq = '5min'
        elif interval == '15min':
            freq = '15min'
        else:
            freq = '1h'

        dates = pd.date_range(end=datetime.now(), periods=periods, freq=freq)

        # Generate realistic BTC price data with random walk
        np.random.seed(42)
        initial_price = 45000  # Starting BTC price

        # Generate returns with some autocorrelation
        returns = np.random.normal(0, 0.02, periods)  # 2% volatility
        returns = np.cumsum(returns)  # Add trend

        prices = initial_price * np.exp(returns)

        # Generate OHLC from close prices
        df = pd.DataFrame({
            'datetime': dates,
            'close': prices
        })

        # Generate realistic OHLC
        df['high'] = df['close'] * (1 + np.abs(np.random.normal(0, 0.005, periods)))
        df['low'] = df['close'] * (1 - np.abs(np.random.normal(0, 0.005, periods)))
        df['open'] = df['close'].shift(1).fillna(df['close'])

        # Ensure OHLC logic
        df['high'] = np.maximum(df['high'], np.maximum(df['open'], df['close']))
        df['low'] = np.minimum(df['low'], np.minimum(df['open'], df['close']))

        # Generate volume
        df['volume'] = np.random.lognormal(15, 1, periods)  # Realistic volume distribution

        print(f"✅ Synthetic {interval} data generated: {len(df)} records")
        return df

    def load_multi_timeframe_data(self):
        """Load data for all timeframes with fallbacks"""
        print("📊 Loading multi-timeframe data...")

        # Test API first
        api_working = self.test_api_connection()

        timeframes = ['5min', '15min', '1h']
        data = {}

        for tf in timeframes:
            print(f"\n🔄 Loading {tf}...")

            # Try main API first
            if api_working:
                df = self.fetch_data(interval=tf, outputsize=2000)
                if df is not None:
                    data[tf] = df
                    continue

            # Try fallback
            print(f"🔄 Trying fallback for {tf}...")
            df = self.fetch_fallback_data(interval=tf)
            if df is not None:
                data[tf] = df
                continue

            # Generate synthetic as last resort
            print(f"🎲 Using synthetic data for {tf}...")
            df = self.generate_synthetic_data(interval=tf)
            data[tf] = df

        print(f"\n✅ Loaded {len(data)} timeframes")
        return data

class SMCPatternDetector:
    """Enhanced Smart Money Concepts pattern detection"""

    @staticmethod
    def detect_market_structure(df, window=20):
        """Detect market structure shifts (BOS/CHoCH)"""
        df = df.copy()

        # Higher Highs/Lower Lows detection
        df['swing_high'] = df['high'].rolling(window, center=True).max() == df['high']
        df['swing_low'] = df['low'].rolling(window, center=True).min() == df['low']

        # Market structure state
        df['market_structure'] = 0  # 0=ranging, 1=bullish, -1=bearish

        swing_highs = df[df['swing_high']]['high'].values
        swing_lows = df[df['swing_low']]['low'].values

        if len(swing_highs) > 1 and len(swing_lows) > 1:
            latest_high = swing_highs[-1] if len(swing_highs) > 0 else 0
            prev_high = swing_highs[-2] if len(swing_highs) > 1 else 0
            latest_low = swing_lows[-1] if len(swing_lows) > 0 else 0
            prev_low = swing_lows[-2] if len(swing_lows) > 1 else 0

            # Break of Structure (BOS)
            if latest_high > prev_high and latest_low > prev_low:
                df.loc[-window:, 'market_structure'] = 1  # Bullish BOS
            elif latest_high < prev_high and latest_low < prev_low:
                df.loc[-window:, 'market_structure'] = -1  # Bearish BOS

        return df['market_structure'].fillna(0)

    @staticmethod
    def detect_liquidity_sweeps(df, lookback=10):
        """Detect liquidity sweeps above/below previous highs/lows"""
        df = df.copy()

        # Recent highs and lows
        df['recent_high'] = df['high'].rolling(lookback).max()
        df['recent_low'] = df['low'].rolling(lookback).min()

        # Liquidity sweep detection
        df['liquidity_sweep'] = 0

        # Sweep above recent high then reject
        high_sweep = (df['high'] > df['recent_high'].shift(1)) & (df['close'] < df['recent_high'].shift(1))
        df.loc[high_sweep, 'liquidity_sweep'] = -1  # Bearish sweep

        # Sweep below recent low then recover
        low_sweep = (df['low'] < df['recent_low'].shift(1)) & (df['close'] > df['recent_low'].shift(1))
        df.loc[low_sweep, 'liquidity_sweep'] = 1  # Bullish sweep

        return df['liquidity_sweep'].fillna(0)

    @staticmethod
    def detect_fair_value_gaps(df, min_gap_percent=0.1):
        """Detect Fair Value Gaps (FVG)"""
        df = df.copy()
        df['fvg_signal'] = 0

        for i in range(2, len(df)):
            # Bullish FVG: gap between candle[i-2].low and candle[i].high
            if (df.iloc[i-2]['low'] > df.iloc[i]['high'] * (1 + min_gap_percent/100) and
                df.iloc[i-1]['close'] > df.iloc[i-1]['open']):  # Middle candle bullish
                df.iloc[i, df.columns.get_loc('fvg_signal')] = 1

            # Bearish FVG: gap between candle[i-2].high and candle[i].low
            elif (df.iloc[i-2]['high'] < df.iloc[i]['low'] * (1 - min_gap_percent/100) and
                  df.iloc[i-1]['close'] < df.iloc[i-1]['open']):  # Middle candle bearish
                df.iloc[i, df.columns.get_loc('fvg_signal')] = -1

        return df['fvg_signal'].fillna(0)

    @staticmethod
    def detect_order_blocks(df, window=5):
        """Detect institutional order blocks"""
        df = df.copy()
        df['order_block'] = 0

        # Bullish order block: strong bullish move after consolidation
        bullish_move = (df['close'] / df['open'] - 1) > 0.01  # 1% move
        recent_range = df['high'].rolling(window).max() - df['low'].rolling(window).min()
        low_volatility = recent_range < df['close'] * 0.005  # Low volatility before move

        bullish_ob = bullish_move & low_volatility.shift(1)
        df.loc[bullish_ob, 'order_block'] = 1

        # Bearish order block: strong bearish move after consolidation
        bearish_move = (df['open'] / df['close'] - 1) > 0.01  # 1% move down
        bearish_ob = bearish_move & low_volatility.shift(1)
        df.loc[bearish_ob, 'order_block'] = -1

        return df['order_block'].fillna(0)

class AdvancedFeatureEngineering:
    """Advanced feature engineering for multi-timeframe analysis"""

    @staticmethod
    def add_technical_indicators(df):
        """Add comprehensive technical indicators"""
        df = df.copy()

        # Price-based features
        df['returns'] = df['close'].pct_change()
        df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
        df['price_range'] = (df['high'] - df['low']) / df['close']
        df['body_size'] = abs(df['close'] - df['open']) / df['close']
        df['upper_shadow'] = (df['high'] - np.maximum(df['open'], df['close'])) / df['close']
        df['lower_shadow'] = (np.minimum(df['open'], df['close']) - df['low']) / df['close']

        # Volume features
        df['volume_sma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / df['volume_sma']
        df['price_volume'] = df['close'] * df['volume']
        df['vwap'] = df['price_volume'].rolling(20).sum() / df['volume'].rolling(20).sum()

        # Volatility features
        df['volatility'] = df['returns'].rolling(20).std()
        df['atr'] = ((df['high'] - df['low']).rolling(14).mean())

        # Momentum indicators
        df['rsi'] = calculate_rsi(df['close'], 14)
        df['macd'], df['macd_signal'] = calculate_macd(df['close'])
        df['bb_upper'], df['bb_lower'] = calculate_bollinger_bands(df['close'])

        # Moving averages
        for period in [5, 10, 20, 50]:
            df[f'sma_{period}'] = df['close'].rolling(period).mean()
            df[f'ema_{period}'] = df['close'].ewm(period).mean()
            df[f'price_sma_{period}_ratio'] = df['close'] / df[f'sma_{period}']

        return df

    @staticmethod
    def add_smc_features(df):
        """Add Smart Money Concepts features"""
        smc = SMCPatternDetector()

        df['market_structure'] = smc.detect_market_structure(df)
        df['liquidity_sweep'] = smc.detect_liquidity_sweeps(df)
        df['fvg_signal'] = smc.detect_fair_value_gaps(df)
        df['order_block'] = smc.detect_order_blocks(df)

        # Displacement detection
        df['displacement'] = 0
        strong_moves = abs(df['returns']) > df['returns'].rolling(50).std() * 2
        df.loc[strong_moves, 'displacement'] = np.sign(df.loc[strong_moves, 'returns'])

        return df

    @staticmethod
    def create_multi_timeframe_features(data_dict, sequence_length=64):
        """Create features from multiple timeframes"""
        print("🔧 Creating multi-timeframe features...")

        # Process each timeframe
        processed_data = {}
        for tf, df in data_dict.items():
            print(f"Processing {tf}...")

            # Add technical indicators
            df = AdvancedFeatureEngineering.add_technical_indicators(df)
            df = AdvancedFeatureEngineering.add_smc_features(df)

            # Clean data
            df = df.fillna(method='ffill').fillna(method='bfill')
            df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

            processed_data[tf] = df

        # Align timeframes (use 15min as base)
        base_df = processed_data['15min'].copy()

        # Add higher timeframe context (1h)
        if '1h' in processed_data:
            hourly_df = processed_data['1h'].copy()
            hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
            hourly_df = hourly_df.set_index('datetime')

            # Resample to 15min and forward fill
            hourly_resampled = hourly_df.resample('15min').ffill()

            # Merge with base timeframe
            base_df['datetime'] = pd.to_datetime(base_df['datetime'])
            base_df = base_df.set_index('datetime')

            # Add hourly features with prefix
            hourly_features = ['market_structure', 'rsi', 'macd', 'volatility', 'price_sma_20_ratio']
            for feature in hourly_features:
                if feature in hourly_resampled.columns:
                    base_df[f'h1_{feature}'] = hourly_resampled[feature]

        # Add lower timeframe signals (5min)
        if '5min' in processed_data:
            min5_df = processed_data['5min'].copy()
            min5_df['datetime'] = pd.to_datetime(min5_df['datetime'])
            min5_df = min5_df.set_index('datetime')

            # Aggregate 5min signals to 15min
            min5_agg = min5_df.resample('15min').agg({
                'liquidity_sweep': 'sum',
                'fvg_signal': 'sum',
                'displacement': 'sum',
                'volatility': 'mean',
                'volume_ratio': 'mean'
            })

            # Add 5min features with prefix
            for col in min5_agg.columns:
                base_df[f'm5_{col}'] = min5_agg[col]

        base_df = base_df.reset_index()
        base_df = base_df.fillna(method='ffill').fillna(0)

        print(f"✅ Multi-timeframe features created: {base_df.shape}")
        return base_df

class EnsembleTradingModel:
    """Ensemble model combining LSTM and XGBoost"""

    def __init__(self, sequence_length=64, save_path=None):
        self.sequence_length = sequence_length
        self.save_path = save_path
        self.lstm_model = None
        self.xgb_model = None
        self.scalers = {}
        self.feature_columns = None

    def load_base_model(self):
        """Load existing v2.0 model as foundation"""
        model_path = os.path.join(self.save_path, 'best_btc_model_v2.keras')

        if os.path.exists(model_path):
            print("📥 Loading base model v2.0...")
            try:
                self.base_model = load_model(model_path)
                print("✅ Base model v2.0 loaded successfully")
                return True
            except Exception as e:
                print(f"❌ Error loading base model: {e}")
                return False
        else:
            print("⚠️ Base model v2.0 not found, will create new model")
            return False

    def create_enhanced_lstm(self, input_shape_15m, input_shape_context=None):
        """Create enhanced LSTM with multi-timeframe inputs"""
        print("🧠 Creating enhanced LSTM architecture...")

        # Main 15min input
        input_15m = Input(shape=input_shape_15m, name='input_15m')

        # LSTM layers for main timeframe (based on v2.0 architecture)
        lstm1 = LSTM(32, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(input_15m)
        lstm1 = BatchNormalization()(lstm1)

        lstm2 = LSTM(16, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)(lstm1)
        lstm2 = BatchNormalization()(lstm2)

        # If we have context features (1h, 5m aggregated)
        if input_shape_context is not None:
            context_input = Input(shape=(input_shape_context,), name='context_input')
            context_dense = Dense(8, activation='relu')(context_input)
            context_dense = Dropout(0.3)(context_dense)

            # Combine LSTM output with context
            combined = Concatenate()([lstm2, context_dense])
        else:
            combined = lstm2

        # Final layers (based on successful v2.0 architecture)
        dense1 = Dense(16, activation='relu')(combined)
        dense1 = BatchNormalization()(dense1)
        dense1 = Dropout(0.5)(dense1)

        output = Dense(3, activation='softmax', name='prediction')(dense1)  # Buy, Hold, Sell

        # Create model
        if input_shape_context is not None:
            model = Model(inputs=[input_15m, context_input], outputs=output)
        else:
            model = Model(inputs=input_15m, outputs=output)

        # Use successful v2.0 optimizer settings
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def prepare_sequences(self, df, target_col='target'):
        """Prepare sequences for LSTM with memory efficiency"""
        print("📊 Preparing sequences...")

        # Select features (excluding non-numeric columns)
        feature_cols = [col for col in df.columns if col not in
                       ['datetime', 'target', 'signal', 'future_return']]

        self.feature_columns = feature_cols

        # Prepare data
        X_data = df[feature_cols].values
        y_data = df[target_col].values if target_col in df.columns else None

        # Scale features
        self.scalers['features'] = StandardScaler()
        X_scaled = self.scalers['features'].fit_transform(X_data)

        # Create sequences
        X_sequences = []
        y_sequences = []

        for i in range(self.sequence_length, len(X_scaled)):
            X_sequences.append(X_scaled[i-self.sequence_length:i])
            if y_data is not None:
                y_sequences.append(y_data[i])

        X_sequences = np.array(X_sequences)

        if y_data is not None:
            y_sequences = np.array(y_sequences)
            print(f"✅ Sequences prepared: {X_sequences.shape}, targets: {y_sequences.shape}")
            return X_sequences, y_sequences
        else:
            print(f"✅ Sequences prepared: {X_sequences.shape}")
            return X_sequences, None

    def create_trading_signals(self, df, profit_threshold=0.004):
        """Create trading signals based on future returns"""
        print(f"🎯 Creating trading signals (threshold: {profit_threshold*100}%)...")

        # Calculate future returns (15 periods ahead for 15min = ~4 hours)
        df['future_return'] = df['close'].shift(-15) / df['close'] - 1

        # Create targets based on v2.0 successful thresholds
        conditions = [
            df['future_return'] > profit_threshold,    # Buy signal
            df['future_return'] < -profit_threshold,   # Sell signal
        ]
        choices = [1, 2]  # 1=Buy, 2=Sell
        df['signal'] = np.select(conditions, choices, default=0)  # 0=Hold

        # Convert to categorical
        from tensorflow.keras.utils import to_categorical
        df['target'] = df['signal']
        y_categorical = to_categorical(df['signal'], num_classes=3)

        # Add categorical columns to dataframe
        df['target_0'] = y_categorical[:, 0]  # Hold
        df['target_1'] = y_categorical[:, 1]  # Buy
        df['target_2'] = y_categorical[:, 2]  # Sell

        signal_counts = df['signal'].value_counts()
        print(f"Signal distribution: {dict(signal_counts)}")

        return df

    def train_ensemble(self, df):
        """Train ensemble model with fine-tuning approach"""
        print("🚀 Training ensemble model...")

        # Create trading signals
        df = self.create_trading_signals(df)

        # Remove rows with NaN targets
        df = df.dropna(subset=['target'])

        # Prepare sequences
        X_sequences, y_sequences = self.prepare_sequences(df, 'target')

        # Train/validation split (80/20 based on v2.0 success)
        split_idx = int(len(X_sequences) * 0.8)

        X_train = X_sequences[:split_idx]
        y_train = y_sequences[:split_idx]
        X_val = X_sequences[split_idx:]
        y_val = y_sequences[split_idx:]

        # Convert targets to categorical
        from tensorflow.keras.utils import to_categorical
        y_train_cat = to_categorical(y_train, num_classes=3)
        y_val_cat = to_categorical(y_val, num_classes=3)

        print(f"Training data: {X_train.shape}, Validation: {X_val.shape}")

        # Create enhanced LSTM
        self.lstm_model = self.create_enhanced_lstm(
            input_shape_15m=(self.sequence_length, X_train.shape[2])
        )

        # Callbacks (based on v2.0 successful settings)
        callbacks = [
            EarlyStopping(patience=15, restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=10, min_lr=0.0001)
        ]

        # Train model
        print("🎓 Training enhanced LSTM...")
        history = self.lstm_model.fit(
            X_train, y_train_cat,
            validation_data=(X_val, y_val_cat),
            epochs=100,
            batch_size=16,  # v2.0 successful batch size
            callbacks=callbacks,
            verbose=1
        )

        # Evaluate
        val_pred = self.lstm_model.predict(X_val)
        val_pred_classes = np.argmax(val_pred, axis=1)
        val_accuracy = accuracy_score(y_val, val_pred_classes)

        print(f"✅ LSTM Validation Accuracy: {val_accuracy:.4f}")

        # Train XGBoost on recent features (memory efficient)
        print("🌲 Training XGBoost...")
        try:
            import xgboost as xgb

            # Use last 2000 samples for XGBoost (memory efficient)
            recent_data = df.tail(2000).copy()
            feature_cols = [col for col in recent_data.columns if col not in
                           ['datetime', 'target', 'signal', 'future_return', 'target_0', 'target_1', 'target_2']]

            X_xgb = recent_data[feature_cols].fillna(0)
            y_xgb = recent_data['target'].values

            # Scale features for XGBoost
            self.scalers['xgb'] = StandardScaler()
            X_xgb_scaled = self.scalers['xgb'].fit_transform(X_xgb)

            # Train XGBoost
            self.xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                random_state=42
            )

            # Train/val split for XGBoost
            split_xgb = int(len(X_xgb_scaled) * 0.8)
            self.xgb_model.fit(
                X_xgb_scaled[:split_xgb],
                y_xgb[:split_xgb]
            )

            # Evaluate XGBoost
            xgb_pred = self.xgb_model.predict(X_xgb_scaled[split_xgb:])
            xgb_accuracy = accuracy_score(y_xgb[split_xgb:], xgb_pred)
            print(f"✅ XGBoost Accuracy: {xgb_accuracy:.4f}")

        except ImportError:
            print("⚠️ XGBoost not available, using LSTM only")
            self.xgb_model = None

        return history

    def predict_ensemble(self, X_sequences, X_features=None):
        """Make ensemble predictions"""
        # LSTM predictions
        lstm_pred = self.lstm_model.predict(X_sequences)

        # XGBoost predictions (if available)
        if self.xgb_model is not None and X_features is not None:
            xgb_pred_proba = self.xgb_model.predict_proba(X_features)

            # Ensure same number of classes
            if xgb_pred_proba.shape[1] == 3:
                # Ensemble averaging
                ensemble_pred = 0.7 * lstm_pred + 0.3 * xgb_pred_proba
            else:
                ensemble_pred = lstm_pred
        else:
            ensemble_pred = lstm_pred

        return ensemble_pred

    def save_models(self):
        """Save all model components"""
        if self.save_path is None:
            return

        print("💾 Saving model components...")

        # Save LSTM model
        lstm_path = os.path.join(self.save_path, 'btc_lstm_v3_multitimeframe.keras')
        self.lstm_model.save(lstm_path)
        print(f"✅ LSTM saved: {lstm_path}")

        # Save ensemble components
        if self.xgb_model is not None:
            ensemble_path = os.path.join(self.save_path, 'btc_ensemble_v3.pkl')
            joblib.dump(self.xgb_model, ensemble_path)
            print(f"✅ XGBoost saved: {ensemble_path}")

        # Save scalers
        scalers_path = os.path.join(self.save_path, 'btc_scalers_v3.pkl')
        joblib.dump(self.scalers, scalers_path)
        print(f"✅ Scalers saved: {scalers_path}")

class MemoryEfficientBacktester:
    """Memory efficient backtesting with chunked processing"""

    def __init__(self, initial_capital=10000, transaction_cost=0.001):
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.results = []

    def backtest_chunked(self, model, df, chunk_size=1000):
        """Perform chunked backtesting to save memory"""
        print("📈 Running memory-efficient backtest...")

        total_chunks = len(df) // chunk_size + 1
        all_results = []

        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size].copy()
            if len(chunk) < model.sequence_length:
                continue

            print(f"Processing chunk {i//chunk_size + 1}/{total_chunks}")

            # Prepare sequences for this chunk
            X_sequences, _ = model.prepare_sequences(chunk)

            if len(X_sequences) == 0:
                continue

            # Make predictions
            predictions = model.lstm_model.predict(X_sequences, verbose=0)
            pred_classes = np.argmax(predictions, axis=1)
            pred_confidence = np.max(predictions, axis=1)

            # Create results for this chunk
            chunk_results = pd.DataFrame({
                'datetime': chunk['datetime'].iloc[model.sequence_length:],
                'close': chunk['close'].iloc[model.sequence_length:],
                'prediction': pred_classes,
                'confidence': pred_confidence,
                'actual_signal': chunk['signal'].iloc[model.sequence_length:] if 'signal' in chunk.columns else 0
            })

            all_results.append(chunk_results)

            # Clean memory
            del X_sequences, predictions, chunk
            gc.collect()

        # Combine all results
        results_df = pd.concat(all_results, ignore_index=True)
        return self.calculate_performance(results_df)

    def calculate_performance(self, results_df, confidence_threshold=0.7):
        """Calculate detailed performance metrics"""
        print("📊 Calculating performance metrics...")

        # Filter by confidence (v2.0 successful approach)
        high_conf = results_df[results_df['confidence'] >= confidence_threshold].copy()

        if len(high_conf) == 0:
            print("⚠️ No high confidence predictions")
            return {}

        # Simulate trading
        capital = self.initial_capital
        position = 0  # 0=no position, 1=long, -1=short
        trades = []
        equity_curve = [capital]

        for i, row in high_conf.iterrows():
            signal = row['prediction']
            price = row['close']

            # Trading logic based on v2.0 success
            if signal == 1 and position <= 0:  # Buy signal
                if position == -1:  # Close short
                    pnl = (entry_price - price) / entry_price * capital * 0.02  # 2% risk
                    capital += pnl * (1 - self.transaction_cost)
                    trades.append({'type': 'close_short', 'price': price, 'pnl': pnl})

                # Open long
                position = 1
                entry_price = price
                trades.append({'type': 'buy', 'price': price})

            elif signal == 2 and position >= 0:  # Sell signal
                if position == 1:  # Close long
                    pnl = (price - entry_price) / entry_price * capital * 0.02  # 2% risk
                    capital += pnl * (1 - self.transaction_cost)
                    trades.append({'type': 'close_long', 'price': price, 'pnl': pnl})

                # Open short
                position = -1
                entry_price = price
                trades.append({'type': 'sell', 'price': price})

            equity_curve.append(capital)

        # Calculate metrics
        total_return = (capital - self.initial_capital) / self.initial_capital

        if len(trades) > 0:
            pnl_trades = [t['pnl'] for t in trades if 'pnl' in t]
            if pnl_trades:
                win_rate = len([p for p in pnl_trades if p > 0]) / len(pnl_trades)
                avg_win = np.mean([p for p in pnl_trades if p > 0]) if any(p > 0 for p in pnl_trades) else 0
                avg_loss = np.mean([p for p in pnl_trades if p < 0]) if any(p < 0 for p in pnl_trades) else 0
            else:
                win_rate = 0
                avg_win = 0
                avg_loss = 0
        else:
            win_rate = 0
            avg_win = 0
            avg_loss = 0

        # Sharpe ratio
        if len(equity_curve) > 1:
            returns = np.diff(equity_curve) / equity_curve[:-1]
            sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252) if np.std(returns) > 0 else 0
        else:
            sharpe_ratio = 0

        # Maximum drawdown
        equity_series = pd.Series(equity_curve)
        rolling_max = equity_series.expanding().max()
        drawdown = (equity_series - rolling_max) / rolling_max
        max_drawdown = drawdown.min()

        # Accuracy vs actual signals
        if 'actual_signal' in high_conf.columns:
            accuracy = accuracy_score(high_conf['actual_signal'], high_conf['prediction'])
        else:
            accuracy = 0

        metrics = {
            'total_return': total_return,
            'accuracy': accuracy,
            'win_rate': win_rate,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'num_trades': len(trades),
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'final_capital': capital,
            'high_confidence_signals': len(high_conf)
        }

        return metrics

# Helper functions for technical indicators
def calculate_rsi(prices, period=14):
    """Calculate RSI"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    return macd.fillna(0), macd_signal.fillna(0)

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    """Calculate Bollinger Bands"""
    sma = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    return upper_band.fillna(sma), lower_band.fillna(sma)

def main():
    """Main execution pipeline"""
    print("🚀 BTC LSTM Trading Model v3.0 - Multi-Timeframe Ensemble")
    print("=" * 60)

    # Initialize components
    data_loader = BTCDataLoader(API_KEY)

    try:
        # Load multi-timeframe data
        data_dict = data_loader.load_multi_timeframe_data()

        if not data_dict:
            print("❌ Failed to load data")
            return

        # Create multi-timeframe features
        df = AdvancedFeatureEngineering.create_multi_timeframe_features(data_dict)

        print(f"📊 Final dataset shape: {df.shape}")
        print(f"Features: {df.columns.tolist()}")

        # Initialize ensemble model
        model = EnsembleTradingModel(sequence_length=64, save_path=SAVE_PATH)

        # Try to load base model v2.0
        model.load_base_model()

        # Train enhanced model
        history = model.train_ensemble(df)

        # Save models
        model.save_models()

        # Memory efficient backtesting
        backtester = MemoryEfficientBacktester()
        metrics = backtester.backtest_chunked(model, df)

        # Display results
        print("\n" + "=" * 60)
        print("📈 BACKTEST RESULTS - BTC LSTM v3.0")
        print("=" * 60)

        print(f"💰 Total Return: {metrics.get('total_return', 0)*100:.2f}%")
        print(f"🎯 Accuracy: {metrics.get('accuracy', 0)*100:.2f}%")
        print(f"✅ Win Rate: {metrics.get('win_rate', 0)*100:.2f}%")
        print(f"📊 Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
        print(f"📉 Max Drawdown: {metrics.get('max_drawdown', 0)*100:.2f}%")
        print(f"🔄 Number of Trades: {metrics.get('num_trades', 0)}")
        print(f"💵 Final Capital: ${metrics.get('final_capital', 0):,.2f}")

        # Compare with v2.0 benchmarks
        print("\n" + "=" * 40)
        print("📊 COMPARISON WITH v2.0")
        print("=" * 40)
        print("v2.0 Results: 66% accuracy, +0.2% return, 0.21 Sharpe")
        print(f"v3.0 Results: {metrics.get('accuracy', 0)*100:.1f}% accuracy, {metrics.get('total_return', 0)*100:+.1f}% return, {metrics.get('sharpe_ratio', 0):.2f} Sharpe")

        improvement = metrics.get('total_return', 0) - 0.002  # v2.0 had +0.2%
        print(f"🚀 Improvement: {improvement*100:+.2f}% return")

        # Save detailed results
        results_path = os.path.join(SAVE_PATH, 'btc_v3_results.csv')
        results_df = pd.DataFrame([metrics])
        results_df.to_csv(results_path, index=False)
        print(f"💾 Results saved: {results_path}")

        print("\n✅ BTC LSTM v3.0 training and evaluation completed!")

        # Memory cleanup
        gc.collect()

    except Exception as e:
        print(f"❌ Error in main execution: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Save path ready: /content/drive/MyDrive/KERAS
🚀 BTC LSTM Trading Model v3.0 - Multi-Timeframe Ensemble
📊 Loading multi-timeframe data...
📡 API Status: {'timestamp': '2025-06-20 21:00:41', 'current_usage': 1, 'plan_limit': 55, 'plan_category': 'grow'}

🔄 Loading 5min...
🔄 Trying symbol: BTCUSD
📡 Response status: 200
📄 Response keys: ['code', 'message', 'status']
📝 API Message: **symbol** or **figi** parameter is missing or invalid. Please provide a valid symbol according to API documentation: https://twelvedata.com/docs#reference-data
🔄 Trying symbol: BTC/USD
📡 Response status: 200
📄 Response keys: ['meta', 'values', 'status']
✅ Raw data shape: (1000, 5)
📋 Columns: ['datetime', 'open', 'high', 'low', 'close']
📊 Created synthetic volume column
✅ 5min data loaded: 1000 records with symbol BTC/USD

🔄 Loading 15min...
🔄 Trying symbol: BTCUSD
📡 Response status: 2

In [ ]:
# ===============================
# REAL-TIME PREDICTION ФУНКЦИЯ
# ===============================

def predict_now():
    """Real-time prediction с свежими данными"""
    print("🔮 REAL-TIME BTC PREDICTION")
    print("=" * 40)

    # Загружаем модель и scalers
    try:
        model_path = os.path.join(SAVE_PATH, 'btc_lstm_v3_multitimeframe.keras')
        scalers_path = os.path.join(SAVE_PATH, 'btc_scalers_v3.pkl')

        from tensorflow.keras.models import load_model
        lstm_model = load_model(model_path)
        scalers = joblib.load(scalers_path)
        print("✅ Модель v3.0 загружена")

        # Загружаем свежие данные
        data_loader = BTCDataLoader(API_KEY)
        df = data_loader.fetch_data('BTCUSD', '15min', outputsize=200)

        if df is None:
            df = data_loader.fetch_fallback_data('15min')

        print(f"📊 Свежие данные: {len(df)} записей")
        print(f"🕐 Последняя свечка: {df['datetime'].iloc[-1]}")
        print(f"💰 Текущая цена BTC: ${df['close'].iloc[-1]:,.2f}")

        # Подготавливаем features
        # Подготавливаем multi-timeframe features как при обучении
        data_dict = {'15min': df}

        # Добавляем 1h контекст
        # Добавляем 1h контекст (с fallback)
        hourly_df = data_loader.fetch_data('BTC/USD', '1h', outputsize=100)
        if hourly_df is None:
            hourly_df = data_loader.fetch_fallback_data('1h')
        if hourly_df is None:
            # Создаем из 15min данных
            hourly_df = df.resample('1h', on='datetime').agg({
                'open': 'first', 'high': 'max', 'low': 'min',
                'close': 'last', 'volume': 'sum'
            }).reset_index()
        data_dict['1h'] = hourly_df

        # Добавляем 5min сигналы (с fallback)
        min5_df = data_loader.fetch_data('BTC/USD', '5min', outputsize=100)
        if min5_df is None:
            min5_df = data_loader.fetch_fallback_data('5min')
        if min5_df is None:
            # Создаем synthetic 5min из 15min
            min5_df = data_loader.generate_synthetic_data('5min', 100)
        data_dict['5min'] = min5_df

        # Создаем те же features что при обучении
        df = AdvancedFeatureEngineering.create_multi_timeframe_features(data_dict)
        df = df.fillna(method='ffill').fillna(0)

        # Берем последние 64 свечки
        # Берем последние 64 свечки
        feature_cols = [col for col in df.columns if col not in
                      ['datetime', 'target', 'signal', 'future_return']]

        X_data = df[feature_cols].tail(64).values

        # Дополняем до нужного количества features (52)
        current_features = X_data.shape[1]
        expected_features = 52

        if current_features < expected_features:
            # Добавляем недостающие features с нулевыми значениями
            missing_features = expected_features - current_features
            padding = np.zeros((X_data.shape[0], missing_features))
            X_data = np.hstack([X_data, padding])
            print(f"🔧 Добавлено {missing_features} features: {current_features} → {expected_features}")
        elif current_features > expected_features:
            # Обрезаем лишние features
            X_data = X_data[:, :expected_features]
            print(f"🔧 Обрезано features: {current_features} → {expected_features}")
        X_scaled = scalers['features'].transform(X_data)
        X_sequence = X_scaled.reshape(1, 64, -1)

        # Prediction
        prediction = lstm_model.predict(X_sequence, verbose=0)[0]
        pred_class = np.argmax(prediction)
        confidence = np.max(prediction)

        # Результат
        signals = {0: "HOLD 📊", 1: "BUY 🟢", 2: "SELL 🔴"}
        signal_name = signals[pred_class]

        print("\n🎯 РЕЗУЛЬТАТ:")
        print(f"🔮 Сигнал: {signal_name}")
        print(f"🎯 Confidence: {confidence*100:.1f}%")
        print(f"📈 Buy: {prediction[1]*100:.1f}%")
        print(f"📊 Hold: {prediction[0]*100:.1f}%")
        print(f"📉 Sell: {prediction[2]*100:.1f}%")

        if confidence >= 0.75:
            print("✅ ВЫСОКАЯ уверенность!")
        elif confidence >= 0.6:
            print("⚠️ СРЕДНЯЯ уверенность")
        else:
            print("❌ НИЗКАЯ уверенность")

        return pred_class, confidence

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return None, None

# Запуск real-time prediction
print("\n" + "🚀 ТЕСТИРУЕМ МОДЕЛЬ НА ЖИВЫХ ДАННЫХ:")
predict_now()


🚀 ТЕСТИРУЕМ МОДЕЛЬ НА ЖИВЫХ ДАННЫХ:
🔮 REAL-TIME BTC PREDICTION
✅ Модель v3.0 загружена
🔄 Trying symbol: BTCUSD
📡 Response status: 200
📄 Response keys: ['code', 'message', 'status']
📝 API Message: **symbol** or **figi** parameter is missing or invalid. Please provide a valid symbol according to API documentation: https://twelvedata.com/docs#reference-data
🔄 Trying symbol: BTC/USD
📡 Response status: 200
📄 Response keys: ['meta', 'values', 'status']
✅ Raw data shape: (200, 5)
📋 Columns: ['datetime', 'open', 'high', 'low', 'close']
📊 Created synthetic volume column
✅ 15min data loaded: 200 records with symbol BTC/USD
📊 Свежие данные: 200 записей
🕐 Последняя свечка: 2025-06-20 21:15:00
💰 Текущая цена BTC: $103,528.21
🔄 Trying symbol: BTC/USD
📡 Response status: 200
📄 Response keys: ['meta', 'values', 'status']
✅ Raw data shape: (100, 5)
📋 Columns: ['datetime', 'open', 'high', 'low', 'close']
📊 Created synthetic volume column
⚠️ Too few records: 100
🔄 Trying symbol: BTC/USD
📡 Response status

ERROR:yfinance:$BTC-USD: possibly delisted; no price data found  (period=1y) (Yahoo error = "5m data not available for startTime=1718918556 and endTime=1750454556. The requested range must be within the last 60 days.")


🎲 Generating synthetic 5min data...
✅ Synthetic 5min data generated: 100 records
🔧 Creating multi-timeframe features...
Processing 15min...
Processing 1h...
Processing 5min...
✅ Multi-timeframe features created: (200, 50)
🔧 Добавлено 3 features: 49 → 52

🎯 РЕЗУЛЬТАТ:
🔮 Сигнал: BUY 🟢
🎯 Confidence: 49.6%
📈 Buy: 49.6%
📊 Hold: 42.9%
📉 Sell: 7.5%
❌ НИЗКАЯ уверенность


(np.int64(1), np.float32(0.49592036))